# 🔎📊🧩 **Exploratory Data Analysis (EDA)**

<img src="./assets/eda_banner.jpg" style="width:75%">

---
---
# 🎯 **1. Introduction** 

This EDA aims to uncover actionable insights into customer no-show behavior for a hotel chain by systematically analyzing the provided dataset. The goal is to identify key factors contributing to no-shows and help formulate policies to mitigate associated costs.

Our analysis will focus on three key areas:

- **Data Overview**: Understanding the structure, characteristics, and quality of the dataset.
- **Feature Analysis**: Identifying variables that significantly influence no-show rates (e.g., booking details, customer demographics, room preferences).
- **Pattern Recognition**: Discovering trends and relationships in customer behavior that could inform strategies to reduce no-shows.

Through comprehensive statistical analysis and visualizations, we will:

- Assess data quality, including handling missing values and verifying assumptions about synthetic features.
- Explore distributions and relationships among features, such as booking month, arrival date, and room type.
- Investigate correlations between customer attributes and no-show events to identify predictive signals.

The insights gained from this EDA will not only inform our predictive modeling approach but also provide valuable understanding of the underlying factors contributing to missed appointments.

🧼 **Data Cleaning Approach**:

Minimal cleaning (e.g., handling missing values, correcting data types, addressing inconsistencies) will be performed during EDA to facilitate exploration. Advanced preprocessing, such as feature engineering and scaling, will be deferred to the machine learning pipeline.

💡 **Key Insights**:

Significant findings uncovered during the analysis will be highlighted using the 💡 emoji. These insights will guide predictive modeling and reveal factors contributing to no-show behavior.

---
---
# 🤔 **2. Preliminary Understanding** 

Before diving into the data, we can form **initial hypotheses** and outline **key areas for investigation** based on the dataset attributes.

These hypotheses and questions will guide our EDA to:

- Identify key predictors of no-show behavior.
- Inform feature engineering and model selection for the ML pipeline.

**📋 Initial Hypothesis**

1. Booking Timing:

   - Longer gaps between `booking_month` and `arrival_month` may reduce no-shows, as customers have more time to plan and commit.
   - Last-minute bookings (`booking_month` close to `arrival_month`) might increase no-show rates due to uncertainty or cancellations.

2. Customer Demographics:

   - First-time customers (`first_time`) may have higher no-show rates compared to returning customers, as they might be less familiar with the hotel’s policies.
   - Customers from specific countries (`country`) might exhibit different no-show behaviors, influenced by travel restrictions, cultural preferences, or booking habits.

3. Room and Pricing:

   - Higher room prices (`price`) might discourage no-shows, as customers are less likely to forfeit expensive bookings.
   - Certain room types (`room`) may attract different customer segments, influencing no-show behavior. For example, luxury rooms might correlate with lower no-show rates.
   
4. Booking Platform:

   - Bookings made through third-party platforms (`platform`) might see higher no-show rates compared to direct bookings, as customers may feel less accountability.

5. Group Size:

   - Larger groups (`num_adults`, `num_children`) may show lower no-show rates, as organizing group travel involves significant planning and commitment.

6. Synthetic Features:

   - We hypothesize that synthetic features may represent engineered variables (e.g., derived from temporal attributes like `booking_month` or `arrival_month`). Their relevance will be verified during EDA.

**📌 Key Areas for Investigation**

- Do seasonal trends (`arrival_month`) affect no-show rates?

- How does booking lead time (`booking_month` vs. `arrival_month`) impact no-show likelihood?

- Are certain platforms (`platform`) or room types (`room`) associated with higher no-show rates?

- What role do customer demographics (`country`, `first_time`) and group size (`num_adults`, `num_children`) play?

---
---
# 📦 **3. Import Dependencies & Setup** 

In [13]:
# Base libraries
import time
import os

# Scientific libraries
import numpy as np
import pandas as pd

# Visual libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Database libraries
import sqlite3

In [56]:
# Visual setup
from matplotlib import rcParams

plt.style.use('ggplot')  # Set the plotting style to 'ggplot' for a clean, professional look.

# Remove the right and top spines (borders) of plots for a minimalist design.
rcParams['axes.spines.right'] = False
rcParams['axes.spines.top'] = False
rcParams['figure.figsize'] = [12, 9]  # Set the default figure size to 12x9 inches for better readability.
rcParams['font.size'] = 16  # Set the base font size for all text elements in the plot to 16.

plt.rc('xtick', labelsize=12)  # Adjust x-axis tick label size to 12 for clarity.
plt.rc('ytick', labelsize=12)  # Adjust y-axis tick label size to 12 for consistency.

# Define a custom color palette for plots with a soft, visually appealing gradient.
custom_colors = ['#74a09e','#86c1b2','#98e2c6','#f3c969','#f2a553', '#d96548', '#c14953']  
sns.set_palette(custom_colors)

%config InlineBackend.figure_format = 'retina'  # Enhance plot resolution for high-quality rendering on Retina displays.
%config Completer.use_jedi = False  # Disable Jedi autocompletion in IPython for faster tab completion.

In [15]:
# Set the maximum width of each column to 40 characters for better readability.
pd.set_option('max_colwidth', 40)

# Display all columns in the output (instead of truncating them with "..." for wide DataFrames).
pd.options.display.max_columns = None  # Can be set to a specific number to limit.

# Configure IPython to display all expressions in a cell, not just the last one.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [57]:
# Seed value for numpy.random for reproducibility
np.random.seed(42)

---
---
# ⏬ **4. Load Data** 

- Establishes a connection to an SQLite database, retrieves the `noshow` table into a DataFrame
- and ensures proper error handling and resource cleanup using a context manager.
- Prepares the dataset for exploratory data analysis (EDA) by loading it into memory as a Pandas DataFrame.

In [22]:
# Load data for EDA
db_path = "data/noshow.db"  # Path to the SQLite database file.

try:
    # Establish a connection to the SQLite database and load the 'noshow' table into a DataFrame.
    with sqlite3.connect(db_path) as conn:
        df = pd.read_sql_query("SELECT * FROM noshow", conn)
    print("✅ Data loaded, connection closed.")  # Confirm successful data loading and connection closure.
except sqlite3.Error as e:
    # Handle and display any database-related errors.
    print(f"❌ Database error: {e}")

✅ Data loaded, connection closed.


In [35]:
# Display the shape of the DataFrame (number of rows and columns) to understand its size.
df.shape  

(119391, 15)

---
---
# 🔎 **5. Basic Exploration** 

This section focuses on gaining an initial understanding of the dataset through various exploratory techniques. By examining the structure, content, and quality of the data, we aim to identify key characteristics, potential issues, and areas requiring further investigation. The steps include:

- **Inspecting Rows**: Displaying the first few (`df.head()`), last few (`df.tail()`), and random rows (`df.sample()`) to understand the dataset's structure and variability.

- **Assessing Data Quality**: Using `df.isnull().sum()` to identify missing values and evaluate data completeness.

- **Summarizing Metadata**: Leveraging `df.info()` to review column names, data types, and non-null counts, ensuring a clear understanding of the dataset's composition.

- **Analyzing Numerical Statistics**: Utilizing `df.describe()` to examine central tendencies, variability, and ranges of numerical features, highlighting potential outliers or anomalies.

These foundational steps provide a comprehensive overview of the dataset, guiding subsequent analysis and preprocessing decisions.

---
### 🔎 **5.1. Display the first few rows**

- Use `df.head()` to display the first few rows of the DataFrame.

- Provides an initial glimpse of the data structure, column names, and sample values.

- Helps verify data loading and understand basic patterns or formats in the dataset.

<br>

---

In [31]:
df.head()  

,booking_id,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children
0,94113,0.0,Changi,November,June,25.0,June,27.0,Singapore,Yes,Single,SGD$ 492.98,Website,1,0.0
1,86543,0.0,Orchard,August,November,28.0,November,29.0,Indonesia,Yes,King,SGD$ 1351.22,Website,2,0.0
2,75928,0.0,Changi,March,February,7.0,February,11.0,India,Yes,Single,None,Agent,1,0.0
3,66947,1.0,Orchard,September,October,1.0,October,3.0,China,Yes,Single,SGD$ 666.04,Website,1,0.0
4,106390,0.0,Orchard,March,June,20.0,June,24.0,Australia,Yes,Queen,USD$ 665.37,Website,1,0.0


💡 **Column Headers**: All column headers are in snake_case, which is ideal for referencing and consistency. No changes required.

💡 **`Price` Column**:

- Includes `SGD$` and `USD$` prefixes (potentially others as well) ➡️ to standardise the currency and remove prefixes to ensure that the data is clean and ready for numerical analysis.

- Contains `None` values (potentially others as well) as missing values ➡️ need to be addressed later, either by imputation, removal, or another appropriate method.

💡 **Preliminary Hypothesis**:

- Based on logical reasoning, certain features can be removed from the dataset due to irrelevance, redundancy, or lack of predictive power. The criteria for removal are as follows:

  - Irrelevance : Features that logically do not contribute to understanding patient no-show behavior (e.g., IDs or unrelated metadata).
  - Redundancy : Features that logically provide duplicate information or are highly correlated with other features.
  - Lack of Predictive Power : Features that, based on logical analysis, show little to no impact on predicting no-show events.

- Potential Features to Remove:

  - `booking_id`: Acts as a unique identifier and does not contribute to predicting no-show behavior.

---
### 🔎 **5.2. Display the last few rows**

- Use `df.tail()` to display the last few rows of the DataFrame.

- Provides a glimpse of the ending records, helping to identify patterns or anomalies toward the end of the dataset.

- Ensures completeness of the data and verifies that all records are loaded correctly.

<br>

---

In [25]:
df.tail()  

,booking_id,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children
119386,4823,0.0,Changi,September,April,3.0,April,5.0,China,Yes,King,USD$ 612.18,Website,1,1.0
119387,25062,0.0,Orchard,March,December,30.0,January,3.0,China,Yes,King,USD$ 1041.29,Email,2,0.0
119388,81936,1.0,Changi,September,March,31.0,April,1.0,Indonesia,Yes,King,USD$ 641.47,Email,1,2.0
119389,6738,0.0,Changi,February,February,25.0,February,27.0,China,Yes,None,SGD$ 886.35,Website,2,0.0
119390,99643,0.0,Changi,March,July,18.0,July,26.0,India,Yes,King,SGD$ 901.95,Website,1,2.0


**💡 Possible Duplicate / Error `booking_id`**

- The last index is 119,390, although there are 113,391 entries.
- To check data later for possible duplicates / some other error.

---
### 🔎 **5.3. Display random samples**

- Use `df.sample()` to display a random sample of rows from the DataFrame.

- Provides an unbiased glimpse of the dataset’s structure and content, ensuring that the sample is not limited to the first or last rows.

- Helps identify patterns, anomalies, or inconsistencies that may not be visible in sequential rows.

<br>

---

In [55]:
df.sample(5, random_state=2508)  # Add random_state for reproducibility

,booking_id,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children
86371,65852,0.0,Changi,February,July,19.0,July,21.0,Japan,Yes,King,USD$ 697.22,Email,2,2.0
30974,8186,0.0,Changi,January,February,12.0,February,14.0,India,Yes,King,SGD$ 890.82,Website,1,2.0
19794,78003,1.0,Changi,January,May,27.0,May,29.0,Australia,Yes,Single,USD$ 402.01,Website,1,0.0
37769,95051,0.0,Changi,August,September,1.0,September,4.0,Indonesia,Yes,King,SGD$ 948.11,Email,1,1.0
32151,113789,1.0,Changi,July,December,21.0,December,23.0,India,Yes,King,USD$ 614.02,Website,two,2.0


💡 **`num_adults` Column**:
- Inconsistent data types detected. (May not be shown when replicated due to sampling)
- Some values are numerical (e.g., `1`), while others are represented as text (e.g., `"one"`). 
- This inconsistency should be addressed by standardizing the format, either converting all values to numerical or textual representation.

---
### 🔎 **5.4. Inspect missing values**

- Use `df.isnull().sum()` to count the number of missing values in each column.

- Identifies columns with missing data, helping to assess data quality and plan for cleaning or imputation.

- Provides insights into potential issues that could affect analysis or modeling if not addressed.

<br>

---

In [6]:
df.isnull().sum()

booking_id            0
no_show               1
branch                1
booking_month         1
arrival_month         1
arrival_day           1
checkout_month        1
checkout_day          1
country               1
first_time            1
room              21613
price             24882
platform              1
num_adults            1
num_children          1
dtype: int64

💡 **Missing Values**:

   - 12 features have 1 missing values each: `no_show`, `branch`, `booking_month`, `arrival_month`, `arrival_day`, `checkout_month`, `checkout_day`, `country`, `first_time`, `platform`, `num_adults`, `num_children` ➡️ Need to check if these are all from the same row or from different rows

   - `room` has 21,613 missing values

   - `price` has 24,882 missing values

---
### 🔎 **5.5. Inspect dataFrame summary**

- Use `df.info()` to display a summary of the DataFrame, including column names, data types, and non-null counts.

- Provides an overview of the dataset's structure, helping to identify missing values and verify data types.

- Ensures all columns are correctly formatted and highlights potential issues that may require cleaning or preprocessing.

<br>

---

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119391 entries, 0 to 119390
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   booking_id      119391 non-null  int64  
 1   no_show         119390 non-null  float64
 2   branch          119390 non-null  object 
 3   booking_month   119390 non-null  object 
 4   arrival_month   119390 non-null  object 
 5   arrival_day     119390 non-null  float64
 6   checkout_month  119390 non-null  object 
 7   checkout_day    119390 non-null  float64
 8   country         119390 non-null  object 
 9   first_time      119390 non-null  object 
 10  room            97778 non-null   object 
 11  price           94509 non-null   object 
 12  platform        119390 non-null  object 
 13  num_adults      119390 non-null  object 
 14  num_children    119390 non-null  float64
dtypes: float64(4), int64(1), object(10)
memory usage: 13.7+ MB


💡 **DataFrame Size**:

   - The DataFrame contains **119,391 rows** and **15 columns**, indicating a moderately sized dataset.

   - The DataFrame consumes **13.7+ MB** of memory, which is reasonable given its size.

💡 **Incorrect Data Types**:

   - The target variable `no_show` is currently as type float64, which is technically correct, but these column should be mapped to categorical values (i.e.1 = Yes and 0 = No) as type object for classification purposes.

   - `price` and `num_adults` are as type objects, but should be converted to float64 and int64 respectively instead

---
### 🔎 **5.6. Inspect Summary Statistics with `df.describe()`**
---

In [60]:
# Display in normal notation instead of scientific
with pd.option_context('float_format', '{:f}'.format):
    # Include all columns in the summary statistics
    df.describe(include='all')

,booking_id,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children
count,119391.000000,119390.000000,119390,119390,119390,119390.000000,119390,119390.000000,119390,119390,97778,94509,119390,119390,119390.000000
unique,NaN,NaN,2,12,88,NaN,12,NaN,7,2,4,61867,4,4,NaN
top,NaN,NaN,Changi,June,August,NaN,August,NaN,China,Yes,King,USD$ 585.23,Website,1,NaN
freq,NaN,NaN,79330,10324,13695,NaN,13836,NaN,48590,115580,70496,11,54234,62745,NaN
mean,59695.000000,0.370416,NaN,NaN,NaN,15.798241,NaN,14.250507,NaN,NaN,NaN,NaN,NaN,NaN,0.871229
std,34465.357332,0.482918,NaN,NaN,NaN,8.780829,NaN,11.063697,NaN,NaN,NaN,NaN,NaN,NaN,0.779796
min,0.000000,0.000000,NaN,NaN,NaN,1.000000,NaN,-31.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
25%,29847.500000,0.000000,NaN,NaN,NaN,8.000000,NaN,7.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
50%,59695.000000,0.000000,NaN,NaN,NaN,16.000000,NaN,15.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
75%,89542.500000,1.000000,NaN,NaN,NaN,23.000000,NaN,23.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000


**💡 Check for Possible Duplicates for `booking_id`** 

   - If complete duplicates (all other values are identical), remove them.

   - If duplicates have different values, clarify with the data collector.

   - If clarification is not possible:
     - Remove entries with incomplete values.
     - Alternatively, retain the most recent entry (prioritizing "show" over "no-show").

**💡 Mean for `no_show` is 0.37**

   - Approximately 63% of customers show up (value = 0) and 37% do not show up (value = 1).

**💡 Invalid Values for `checkout_day`** 

   - The minimum value is -31, which is invalid as the day of the month cannot be negative. 
   
   - This could be an error during data collection.

   - Verify with the data collector if this is a mistake. If confirmed as an error, take the absolute value (abs(`checkout_day`)).

   - If clarification is not possible:
     - Count occurrences of negative values.
     - If the occurrences are minimal relative to the dataset size, consider removing these entries.

**💡 Invalid Data Types for `Price` and `num_adults`**

   - These columns need to be cleaned and converted to numerical formats for further analysis.
   
   - Investigate and resolve formatting inconsistencies before proceeding with statistical analysis or modeling.

---
---
# 🧼 **6. Data Cleaning**

Issue List for the Dataset:

- Irrelevant features: `booking_id` ➡️ To remove
- 

In [44]:
# For reference
df.head(1)

,booking_id,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children
0,94113,0.0,Changi,November,June,25.0,June,27.0,Singapore,Yes,Single,SGD$ 492.98,Website,1,0.0


---
---
# 📈 **7. Univariate Exploration**

---
---
# 📊 **8. Bivariate Exploration**

---
---
# ♻️ **9. Multivariate Exploration**

---
---
# 🚀 **10. Conclusion & Next Steps**

- General cleaning (Carried out above for better exploration, need to replicate in the e2e pipeline)
- Preprocessing steps (To be added to the pipeline)
- 